In [57]:
import os
import random
import metpy
import datetime
import s3fs
import copy
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import scipy.stats

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

In [3]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37] == x[27:41]:
            return x

In [9]:
def control_img(img):
    """
    Returns a dictionary of errors for the given file without sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to evaluate the error of
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    return control

def laplace_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using laplace sharpening.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + 0.8*laplace_edges
    return sharpimg

def unsharpmask_sharpening_img(img):
    """
    Returns a dictionary of errors for the given file using unsharpmask sharpening. To normalize images,
    each image is divided by its maximum value, and then multiplied by the same value after sharpening
    is completed.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a str
    """
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    return sharpimg

def update_dict_img(fires, cloud_mask, img07, img14, data1, data2, data3, data4):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    data07 = control_img(img07)
    data14 = control_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data1['no_of_fires'].append(np.count_nonzero(datafires))
    data1['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data1['TPR'].append(otherstats[2])
    data1['PPV'].append(otherstats[3])
    
    data07 = laplace_sharpening_img(img07)
    data14 = laplace_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data2['no_of_fires'].append(np.count_nonzero(datafires))
    data2['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data2['TPR'].append(otherstats[2])
    data2['PPV'].append(otherstats[3])
    
    data07 = unsharpmask_sharpening_img(img07)
    data14 = unsharpmask_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data3['no_of_fires'].append(np.count_nonzero(datafires))
    data3['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data3['TPR'].append(otherstats[2])
    data3['PPV'].append(otherstats[3])
    
    data07 = esrgan_sharpening_img(img07)
    data14 = esrgan_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data4['no_of_fires'].append(np.count_nonzero(datafires))
    data4['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data4['TPR'].append(otherstats[2])
    data4['PPV'].append(otherstats[3])
    
def fire_threshold(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    points = []
    
    fire_mask = (img14) < (m*img07 + b)
    points = np.array(fire_mask.nonzero()).T
    
    for point in points:
        save = point[0]
        point[0] = point[1]
        point[1] = 500 - save
        
    return points

def fire_mask(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    return (img14) < (m*img07 + b)

def dice_score(x, y):
    numerator = 2*np.count_nonzero(np.logical_and(x, y))
    denominator = np.count_nonzero(x) + np.count_nonzero(y)
    return numerator/denominator

def other_stats(x, y):
    false_positives = np.logical_and(np.logical_not(x), y)
    false_negatives = np.logical_and(x, np.logical_not(y))
    #true positive rate
    TPR = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(x)
    #positive predictive value
    try:
        PPV = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(y)
    except ZeroDivisionError:
        PPV = float('NaN')
    return [np.count_nonzero(false_positives), np.count_nonzero(false_negatives), TPR, PPV]

In [5]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
    height, width = image.shape
    result = np.zeros((image.shape[0]//2, image.shape[1]//2, 3))
    smallimg = resize(image, (round(height/2), round(width/2)))
    result[:,:,0] = smallimg
    result[:,:,1] = smallimg
    result[:,:,2] = smallimg
    lr_image = tf.expand_dims(result, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

def esrgan_sharpening_img(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))
    lr_image = downscale_image(image)
    fake_image = model(lr_image)
    fake_image = tf.squeeze(fake_image)

    hr_image = tf.squeeze(hr_image).numpy()
    lr_image = tf.squeeze(lr_image).numpy()
    lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
    fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3))

    return np.mean(fake_image, axis=2)

# Error

In [15]:
filedates = set()
files = []

for file in os.listdir('../../GOES_Files/npy_files'):
    filedates.add(file[27:41])

for file in os.listdir('../../GOES_Files/clear_sky_mask'):
    if file[23:37] in filedates:
        files.append(file)

In [16]:
mean = []
std = []
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
fires_initial = []
fires_final_control = {'no_of_fires':[], 'dice_score':[], 'TPR':[], 'PPV':[]}
fires_final_lp = copy.deepcopy(fires_final_control)
fires_final_um = copy.deepcopy(fires_final_control)
fires_final_ml = copy.deepcopy(fires_final_control)

x = 0

while x < 100:
    try:
        CM = random.choice(files)
        files.remove(CM)

        file07 = find_npy_file('../../GOES_Files/npy_files', CM, '07')
        file14 = find_npy_file('../../GOES_Files/npy_files', CM, '14')
        img07 = np.load(path + file07)
        img14 = np.load(path + file14)

        cloud_mask = np.load(pathCM + CM)
        cloud_mask = cloud_mask.astype(bool)
        fires = np.logical_and(fire_mask(img07, img14), np.logical_not(cloud_mask))
        
        if np.count_nonzero(fires) != 0:
            fires_initial.append(np.count_nonzero(fires))
            update_dict_img(fires, cloud_mask, img07, img14, fires_final_control, fires_final_lp, fires_final_um, fires_final_ml)

            x += 1

    except:
        pass

In [51]:
def results(stat):
    data = [fires_final_control, fires_final_lp, fires_final_um, fires_final_ml]
    datanames = ['Control', 'Laplace', 'Unsharp Mask', 'ESRGAN']
    print(stat)
    for x in range(len(data)):
        print(f'     {datanames[x]} {round(np.nanmean(data[x][stat]), 4)} +/- {round(2*np.nanstd(data[x][stat]), 4)}')

In [56]:
print(round(np.mean(fires_initial), 4))
results('no_of_fires')

9.8911
no_of_fires
     Control 9.5446 +/- 27.876
     Laplace 12.1188 +/- 30.8048
     Unsharp Mask 10.7624 +/- 29.0144
     ESRGAN 9.92 +/- 26.6664


In [53]:
results('dice_score')

dice_score
     Control 0.7126 +/- 0.6447
     Laplace 0.7444 +/- 0.4975
     Unsharp Mask 0.7301 +/- 0.5894
     ESRGAN 0.8106 +/- 0.5634


In [54]:
results('TPR')

TPR
     Control 0.7294 +/- 0.706
     Laplace 0.8614 +/- 0.5342
     Unsharp Mask 0.7914 +/- 0.6486
     ESRGAN 0.8333 +/- 0.586


In [52]:
results('PPV')

PPV
     Control 0.8154 +/- 0.4647
     Laplace 0.6929 +/- 0.4883
     Unsharp Mask 0.7552 +/- 0.4815
     ESRGAN 0.8538 +/- 0.417


In [64]:
scipy.stats.ttest_ind(fires_final_lp['dice_score'], fires_final_ml['dice_score'])

Ttest_indResult(statistic=-1.7562955738269859, pvalue=0.08057619334530504)